## Motivation

In this notebook, we exame the idea declared in section 1.5.5 on MNIST data, a classification task.

In [1]:
import numpy as np
import tensorflow as tf
from keras.losses import MSE
from sklearn.metrics import accuracy_score

from utils import get_gradient_loss_fn

tf.random.set_seed(42)

2024-03-19 18:43:00.811182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## The MNIST Data

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32').reshape([-1, 28*28]) / 255.0
x_test = x_test.astype('float32').reshape([-1, 28*28]) / 255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

## Train a Model with Gradient Loss

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, 'relu'),
    tf.keras.layers.Dense(128, 'relu'),
    tf.keras.layers.Dense(10, 'softmax')
])

gradient_loss_fn = get_gradient_loss_fn(
    lambda inputs: MSE(inputs[1], model(inputs[0]))
)

In [4]:
optimizer = tf.optimizers.Adam()

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        loss = gradient_loss_fn((x, y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    return loss

In [5]:
def evaluate(model):
    return accuracy_score(y_test, tf.argmax(model(x_test), axis=1))

In [6]:
ds = tf.data.Dataset.from_tensor_slices(
    (x_train, tf.one_hot(y_train, 10))
)
ds = ds.batch(100)

In [7]:
for epoch in range(20):
    for x, y in ds:
        loss = train_step(x, y)
    print(epoch, loss)

0 tf.Tensor(0.00026875423, shape=(), dtype=float32)
1 tf.Tensor(0.00036284144, shape=(), dtype=float32)
2 tf.Tensor(0.00018488847, shape=(), dtype=float32)
3 tf.Tensor(0.000107098764, shape=(), dtype=float32)
4 tf.Tensor(0.00010771234, shape=(), dtype=float32)
5 tf.Tensor(0.00012284383, shape=(), dtype=float32)
6 tf.Tensor(8.736429e-05, shape=(), dtype=float32)
7 tf.Tensor(9.960141e-05, shape=(), dtype=float32)
8 tf.Tensor(8.0831975e-05, shape=(), dtype=float32)
9 tf.Tensor(8.0603975e-05, shape=(), dtype=float32)
10 tf.Tensor(8.287506e-05, shape=(), dtype=float32)
11 tf.Tensor(8.090177e-05, shape=(), dtype=float32)
12 tf.Tensor(8.006299e-05, shape=(), dtype=float32)
13 tf.Tensor(8.036914e-05, shape=(), dtype=float32)
14 tf.Tensor(0.00012589067, shape=(), dtype=float32)
15 tf.Tensor(8.003903e-05, shape=(), dtype=float32)
16 tf.Tensor(8.094287e-05, shape=(), dtype=float32)
17 tf.Tensor(8.0417e-05, shape=(), dtype=float32)
18 tf.Tensor(8.194218e-05, shape=(), dtype=float32)
19 tf.Tensor(8

In [8]:
evaluate(model)

0.9747

## Baseline Model with Usual Loss

In [9]:
baseline_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, 'softmax')
])

In [10]:
baseline_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [11]:
baseline_model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_test, y_test),
)

Epoch 1/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2334 - accuracy: 0.9315 - val_loss: 0.1165 - val_accuracy: 0.9634
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0985 - accuracy: 0.9696 - val_loss: 0.1174 - val_accuracy: 0.9632
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0688 - accuracy: 0.9783 - val_loss: 0.0762 - val_accuracy: 0.9758
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0510 - accuracy: 0.9830 - val_loss: 0.0937 - val_accuracy: 0.9727
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0404 - accuracy: 0.9863 - val_loss: 0.1026 - val_accuracy: 0.9719
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0323 - accuracy: 0.9895 - val_loss: 0.0806 - val_accuracy: 0.9757
Epoch 7/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0276 - accuracy: 0.9912 - val_loss: 0.1060 - val_accuracy:

In [12]:
evaluate(baseline_model)

0.9809

## Model Robustness

Now, we compare the robustness of the model and the baseline. To do so, we add Gaussian noise to the test data and check the accuracy.

In [13]:
stddev = 3e-1
noise = tf.random.normal(tf.shape(x_test)) * stddev

In [14]:
def evaluate_robustness(model):
    accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test), axis=1))
    noised_accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test+noise), axis=1))
    print(f'Accuracy: {accuracy} -> {noised_accuracy}')

In [15]:
evaluate_robustness(model)

Accuracy: 0.9747 -> 0.9341


In [16]:
evaluate_robustness(baseline_model)

Accuracy: 0.9809 -> 0.9062


## Conclusion

By simply using the "gradient loss", we obtain a result that approaches the baseline. But the robustness is greatly out-performs the baseline.